In [ ]:
## Robust Object Detection and Tracking (Webcam): ##

# Real-world scenario:
# Detect and track a colored object (for example a red cap, tape, or marker) reliably under changing lighting, while filtering noise and recording the session.

"""
    A stable system usually follows this flow:

1.Capture frame
2.Convert to HSV
3.Threshold color
4.Remove noise
5.Filter by area
6.Track centroid
7.Show FPS
8.Record video

    Each stage fixes a different failure mode.
"""

'\n    A stable system usually follows this flow:\n\n1.Capture frame\n2.Convert to HSV\n3.Threshold color\n4.Remove noise\n5.Filter by area\n6.Track centroid\n7.Show FPS\n8.Record video\n\n    Each stage fixes a different failure mode.\n'

In [ ]:
# WORKING DEMO 01

import cv2
import numpy as np
import time

cap = cv2.VideoCapture(0)

# Video recorder
#fourcc = cv2.VideoWriter_fourcc(*'XVID')
#out = cv2.VideoWriter('demo_output.avi', fourcc, 20.0, (640,480))

prev_time = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (640,480))

    # Convert to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Red color range (can tune)
    lower_red = np.array([0,120,70])
    upper_red = np.array([10,255,255])

    mask = cv2.inRange(hsv, lower_red, upper_red)

    # Noise removal
    kernel = np.ones((5,5),np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

    # Find contours
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for cnt in contours:
        area = cv2.contourArea(cnt)

        # False positive handling
        if area > 800:
            x,y,w,h = cv2.boundingRect(cnt)

            # Draw box
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)

            # Find centroid
            M = cv2.moments(cnt)
            if M["m00"] != 0:
                cx = int(M["m10"]/M["m00"])
                cy = int(M["m01"]/M["m00"])
                cv2.circle(frame,(cx,cy),5,(0,0,255),-1)

    # FPS Calculation
    current_time = time.time()
    fps = 1/(current_time-prev_time) if prev_time!=0 else 0
    prev_time = current_time

    cv2.putText(frame,f"FPS:{int(fps)}",(10,30),
                cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,255,0),2)

    # Show
    cv2.imshow("Mask", mask)
    cv2.imshow("Detection", frame)

    # Record demo
    #out.write(frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
#out.release()
cv2.destroyAllWindows()


# PROBLEM:
# The Display text for FPS is too fast and inacurate, for a cam of 30fps its showing 50fps too.

In [ ]:
# WORKING DEMO 02

# SOLUTION:
# To stop the FPS text from flickering like a glitchy strobe light, you need to dampen the update frequency.
# Use a counter or a timer to only update the displayed text every 15 frames.
# You need to divide the number of frames that passed (15) by the total time elapsed.

import cv2
import numpy as np
import time

cap = cv2.VideoCapture(0)

# Video recorder
#fourcc = cv2.VideoWriter_fourcc(*'XVID')
#out = cv2.VideoWriter('demo_output.avi', fourcc, 20.0, (640,480))

# --- INITIALIZE THESE BEFORE THE WHILE LOOP ---
prev_time = time.time()  # Start the clock
fps_display = 0
frame_count = 0
update_interval = 15     # Update text every 15 frames

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (640,480))

    # Convert to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Red color range (can tune)
    lower_red = np.array([0,120,70])
    upper_red = np.array([10,255,255])

    mask = cv2.inRange(hsv, lower_red, upper_red)

    # Noise removal
    kernel = np.ones((5,5),np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

    # Find contours
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for cnt in contours:
        area = cv2.contourArea(cnt)

        # False positive handling
        if area > 800:
            x,y,w,h = cv2.boundingRect(cnt)

            # Draw box
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)

            # Find centroid
            M = cv2.moments(cnt)
            if M["m00"] != 0:
                cx = int(M["m10"]/M["m00"])
                cy = int(M["m01"]/M["m00"])
                cv2.circle(frame,(cx,cy),5,(0,0,255),-1)

    if frame_count >= update_interval:
        current_time = time.time()
        elapsed_time = current_time - prev_time
        
        # Calculate actual FPS: (Number of Frames) / (Seconds passed)
        fps_display = frame_count / elapsed_time
        
        # Reset for the next batch
        prev_time = current_time
        frame_count = 0
        
    cv2.putText(frame,f"FPS:{int(fps)}",(10,30),
                cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,255,0),2)

    # Show
    cv2.imshow("Mask", mask)
    cv2.imshow("Detection", frame)

    # Record demo
    #out.write(frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
#out.release()
cv2.destroyAllWindows()


# PROBLEM:
# Cam limit is 30fps, but sometimes its showing 35 fps
# Reason: Interanl Buffering (cap.read() sometimes returns "cached frames"), & The waitKey(1) "Short circuit"


In [ ]:
# WOKING DEMO 03

# SOLUTION:
# To get a truthful number, you should calculate the Average FPS over a fixed period of time (like every 30 frames) instead of a single frame.
# By measuring over a full second, you smooth out the tiny hardware/software hiccups that cause "fake" 35 FPS spikes. This gives you the Actual Processing Rate of your system.

import cv2
import numpy as np
import time

cap = cv2.VideoCapture(0)

# Video recorder
#fourcc = cv2.VideoWriter_fourcc(*'XVID')
#out = cv2.VideoWriter('demo_output.avi', fourcc, 20.0, (640,480))

# --- INITIALIZE THESE BEFORE THE WHILE LOOP ---
frame_count = 0
fps_display = 0
start_time = time.time()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (640,480))

    # Convert to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Red color range (can tune)
    lower_red = np.array([0,120,70])
    upper_red = np.array([10,255,255])

    mask = cv2.inRange(hsv, lower_red, upper_red)

    # Noise removal
    kernel = np.ones((5,5),np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

    # Find contours
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for cnt in contours:
        area = cv2.contourArea(cnt)

        # False positive handling
        if area > 800:
            x,y,w,h = cv2.boundingRect(cnt)

            # Draw box
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)

            # Find centroid
            M = cv2.moments(cnt)
            if M["m00"] != 0:
                cx = int(M["m10"]/M["m00"])
                cy = int(M["m01"]/M["m00"])
                cv2.circle(frame,(cx,cy),5,(0,0,255),-1)

    # Update stats every 1 second
    current_time = time.time()
    elapsed = current_time - start_time
    
    if elapsed >= 1.0: # 1 second has passed
        fps_display = frame_count / elapsed
        frame_count = 0
        start_time = current_time
        
    cv2.putText(frame,f"FPS:{int(fps)}",(10,30),
                cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,255,0),2)

    # Show
    cv2.imshow("Mask", mask)
    cv2.imshow("Detection", frame)

    # Record demo
    #out.write(frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
#out.release()
cv2.destroyAllWindows()

In [ ]:
# WORKING DEMO 4

# Add Median BLur
# Standard blurs (like Gaussian) average the pixels together, which can "smear" the edges of your object. A Median Blur replaces each pixel with the median value in its neighborhood. This is excellent for:
# (i) Removing small speckles without blurring the sharp edges of your red object.
# (ii) Making the mask "solid" so the contour detection doesn't see one object as three small pieces.

# Update the "Red Mask"
# Since you are tracking Red, your current code has a weakness. In HSV, "Red" exists at both ends of the spectrum (0–10 and 170–180). To make your tracking truly robust, you should capture both ranges.


import cv2
import numpy as np
import time

cap = cv2.VideoCapture(0)

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# Video recorder
#fourcc = cv2.VideoWriter_fourcc(*'mp4v')
#out = cv2.VideoWriter('Red_Object_Tracking_Demo.mp4v', fourcc, 20.0, (640,480))

frame_count = 0
fps_display = 0
start_time = time.time()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (640,480))

    # Convert to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # The number (5) must be an odd integer. Higher = smoother/slower.
    hsv = cv2.medianBlur(hsv, 5)

    #  In HSV, "Red" exists at both ends of the spectrum (0–10 and 170–180). To make your tracking truly robust, you should capture both ranges:
    # Range for lower red
    lower_red1 = np.array([0, 120, 70])
    upper_red1 = np.array([10, 255, 255])
    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)

    # Range for upper red (near 180)
    lower_red2 = np.array([170, 120, 70])
    upper_red2 = np.array([180, 255, 255])
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)

    # Combine them
    mask = cv2.bitwise_or(mask1, mask2)
    
    
    # Noise removal
    kernel = np.ones((5,5),np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

    # Find contours
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for cnt in contours:
        area = cv2.contourArea(cnt)

        # False positive handling
        if area > 800:
            x,y,w,h = cv2.boundingRect(cnt)

            # Draw box
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)

            # Find centroid
            M = cv2.moments(cnt)
            if M["m00"] != 0:
                cx = int(M["m10"]/M["m00"])
                cy = int(M["m01"]/M["m00"])
                cv2.circle(frame,(cx,cy),5,(0,0,255),-1)

   # FPS Calculation logic
    frame_count += 1
    
    # Update stats every 1 second
    current_time = time.time()
    elapsed = current_time - start_time
    
    if elapsed >= 1.0: # 1 second has passed
        fps_display = frame_count / elapsed
        frame_count = 0
        start_time = current_time
        
    # Use the 'fps_display' variable so the text stays still for a moment
    cv2.putText(frame, f"FPS: {int(fps_display)}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
   
    # Show
    cv2.imshow("Mask", mask)
    cv2.imshow("Detection", frame)

    # Record demo
    #out.write(frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
#out.release()
cv2.destroyAllWindows()

In [2]:
# WORKING DEMO 5

# Tune HSV using trackbars


import cv2
import numpy as np
import time

cap = cv2.VideoCapture(0)

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# Video recorder
#fourcc = cv2.VideoWriter_fourcc(*'mp4v')
#out = cv2.VideoWriter('Red_Object_Tracking_Demo.mp4v', fourcc, 20.0, (640,480))

def nothing(x):
    pass

# 1. Create the window with the 'NORMAL' flag (allows resizing)
cv2.namedWindow("Tuner", cv2.WINDOW_NORMAL)

# 2. Provide your own size (Width, Height)
# A height of 400-500 is usually perfect for 8 vertical trackbars.
cv2.resizeWindow("Tuner", 600, 500) 

# 3. NOW add your trackbarsrols
cv2.namedWindow("Tuner")
cv2.createTrackbar("L-H", "Tuner", 0, 179, nothing)
cv2.createTrackbar("L-S", "Tuner", 120, 255, nothing)
cv2.createTrackbar("L-V", "Tuner", 70, 255, nothing)
cv2.createTrackbar("U-H", "Tuner", 10, 179, nothing)
cv2.createTrackbar("U-S", "Tuner", 255, 255, nothing)
cv2.createTrackbar("U-V", "Tuner", 255, 255, nothing)
dummy_img = np.zeros((1, 500, 3), np.uint8) # Create a tiny 1-pixel high image

frame_count = 0
fps_display = 0
start_time = time.time()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (640,480))

    # Convert to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # The number (5) must be an odd integer. Higher = smoother/slower.
    hsv = cv2.medianBlur(hsv, 5)

    # Get current positions of trackbars
    l_h = cv2.getTrackbarPos("L-H", "Tuner")
    l_s = cv2.getTrackbarPos("L-S", "Tuner")
    l_v = cv2.getTrackbarPos("L-V", "Tuner")
    u_h = cv2.getTrackbarPos("U-H", "Tuner")
    u_s = cv2.getTrackbarPos("U-S", "Tuner")
    u_v = cv2.getTrackbarPos("U-V", "Tuner")

    # Update ranges dynamically
    lower_red = np.array([l_h, l_s, l_v])
    upper_red = np.array([u_h, u_s, u_v])

    mask = cv2.inRange(hsv, lower_red, upper_red)
    
    
    # Noise removal
    kernel = np.ones((5,5),np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

    # Find contours
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for cnt in contours:
        area = cv2.contourArea(cnt)

        # False positive handling
        if area > 800:
            x,y,w,h = cv2.boundingRect(cnt)

            # Draw box
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)

            # Find centroid
            M = cv2.moments(cnt)
            if M["m00"] != 0:
                cx = int(M["m10"]/M["m00"])
                cy = int(M["m01"]/M["m00"])
                cv2.circle(frame,(cx,cy),5,(0,0,255),-1)

   # FPS Calculation logic
    frame_count += 1
    
    # Update stats every 1 second
    current_time = time.time()
    elapsed = current_time - start_time
    
    if elapsed >= 1.0: # 1 second has passed
        fps_display = frame_count / elapsed
        frame_count = 0
        start_time = current_time
        
    # Use the 'fps_display' variable so the text stays still for a moment
    cv2.putText(frame, f"FPS: {int(fps_display)}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
   
    # Show
    cv2.imshow("Mask", mask)
    cv2.imshow("Detection", frame)
    cv2.imshow("Tuner", dummy_img) # This forces the vertical stack

    # Record demo
    #out.write(frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
#out.release()
cv2.destroyAllWindows()

In [4]:
# WORKING DEMO 6

# Dual Mask Tuning of HSV using trackbars


import cv2
import numpy as np
import time

cap = cv2.VideoCapture(0)

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# Video recorder
#fourcc = cv2.VideoWriter_fourcc(*'mp4v')
#out = cv2.VideoWriter('Red_Object_Tracking_Demo.mp4v', fourcc, 20.0, (640,480))


def nothing(x):
    pass

# 1. Create the window with the 'NORMAL' flag (allows resizing)
cv2.namedWindow("Tuner", cv2.WINDOW_NORMAL)

# 2. Provide your own size (Width, Height)
# A height of 400-500 is usually perfect for 8 vertical trackbars.
cv2.resizeWindow("Tuner", 600, 500) 

# 3. NOW add your trackbars
cv2.createTrackbar("Low Hue 1", "Tuner", 0, 179, nothing)
cv2.createTrackbar("Up Hue 1", "Tuner", 10, 179, nothing)
cv2.createTrackbar("Low Hue 2", "Tuner", 170, 179, nothing)
cv2.createTrackbar("Up Hue 2", "Tuner", 180, 179, nothing)
cv2.createTrackbar("Min Sat", "Tuner", 120, 255, nothing)
cv2.createTrackbar("Min Val", "Tuner", 70, 255, nothing)
cv2.createTrackbar("Max Sat", "Tuner", 255, 255, nothing)
cv2.createTrackbar("Max Val", "Tuner", 255, 255, nothing)

frame_count = 0
fps_display = 0
start_time = time.time()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (640,480))

    # Convert to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # The number (5) must be an odd integer. Higher = smoother/slower.
    hsv = cv2.medianBlur(hsv, 5)

    # Get Trackbar positions
    lh1, uh1 = cv2.getTrackbarPos("Low Hue 1", "Tuner"), cv2.getTrackbarPos("Up Hue 1", "Tuner")
    lh2, uh2 = cv2.getTrackbarPos("Low Hue 2", "Tuner"), cv2.getTrackbarPos("Up Hue 2", "Tuner")
    ls, lv = cv2.getTrackbarPos("Min Sat", "Tuner"), cv2.getTrackbarPos("Min Val", "Tuner")
    us, uv = cv2.getTrackbarPos("Max Sat", "Tuner"), cv2.getTrackbarPos("Max Val", "Tuner")

    # Dual Mask Logic
    mask1 = cv2.inRange(hsv, np.array([lh1, ls, lv]), np.array([uh1, us, uv]))
    mask2 = cv2.inRange(hsv, np.array([lh2, ls, lv]), np.array([uh2, us, uv]))
    mask = cv2.bitwise_or(mask1, mask2)
    
    
    # Noise removal
    kernel = np.ones((5,5),np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

    # Find contours
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for cnt in contours:
        area = cv2.contourArea(cnt)

        # False positive handling
        if area > 800:
            x,y,w,h = cv2.boundingRect(cnt)

            # Draw box
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)

            # Find centroid
            M = cv2.moments(cnt)
            if M["m00"] != 0:
                cx = int(M["m10"]/M["m00"])
                cy = int(M["m01"]/M["m00"])
                cv2.circle(frame,(cx,cy),5,(0,0,255),-1)

   # FPS Calculation logic
    frame_count += 1
    
    # Update stats every 1 second
    current_time = time.time()
    elapsed = current_time - start_time
    
    if elapsed >= 1.0: # 1 second has passed
        fps_display = frame_count / elapsed
        frame_count = 0
        start_time = current_time
        
    # Use the 'fps_display' variable so the text stays still for a moment
    cv2.putText(frame, f"FPS: {int(fps_display)}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
   
    # Show
    cv2.imshow("Mask", mask)
    cv2.imshow("Detection", frame)

    # Record demo
    #out.write(frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
#out.release()
cv2.destroyAllWindows()

In [3]:
# WORKING DEMO 7

# Optimized Code with Dual Masking & Persistent Tuning

import cv2
import numpy as np
import json
import os

# --- FILE HANDLING ---
CONFIG_FILE = "hsv_settings.json"

def save_settings(values):
    with open(CONFIG_FILE, 'w') as f:
        json.dump(values, f)

def load_settings():
    if os.path.exists(CONFIG_FILE):
        with open(CONFIG_FILE, 'r') as f:
            return json.load(f)
    return {"lh1": 0, "uh1": 10, "lh2": 170, "uh2": 180, "ls": 120, "lv": 70, "us": 255, "uv": 255}

# --- SETUP ---
def nothing(x): pass
vals = load_settings()

# 1. Create the window with the 'NORMAL' flag (allows resizing)
cv2.namedWindow("Tuner", cv2.WINDOW_NORMAL)

# 2. Provide your own size (Width, Height)
# A height of 400-500 is usually perfect for 8 vertical trackbars.
cv2.resizeWindow("Tuner", 600, 500) 

# 3. NOW add your trackbars
cv2.createTrackbar("Low Hue 1", "Tuner", vals["lh1"], 179, nothing)
cv2.createTrackbar("Up Hue 1", "Tuner", vals["uh1"], 179, nothing)
cv2.createTrackbar("Low Hue 2", "Tuner", vals["lh2"], 179, nothing)
cv2.createTrackbar("Up Hue 2", "Tuner", vals["uh2"], 179, nothing)
cv2.createTrackbar("Min Sat", "Tuner", vals["ls"], 255, nothing)
cv2.createTrackbar("Min Val", "Tuner", vals["lv"], 255, nothing)
cv2.createTrackbar("Max Sat", "Tuner", vals["us"], 255, nothing)
cv2.createTrackbar("Max Val", "Tuner", vals["uv"], 255, nothing)

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Pre-processing
    frame = cv2.flip(frame, 1)
    blurred = cv2.medianBlur(frame, 5)
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)

    # Get Trackbar positions
    lh1, uh1 = cv2.getTrackbarPos("Low Hue 1", "Tuner"), cv2.getTrackbarPos("Up Hue 1", "Tuner")
    lh2, uh2 = cv2.getTrackbarPos("Low Hue 2", "Tuner"), cv2.getTrackbarPos("Up Hue 2", "Tuner")
    ls, lv = cv2.getTrackbarPos("Min Sat", "Tuner"), cv2.getTrackbarPos("Min Val", "Tuner")
    us, uv = cv2.getTrackbarPos("Max Sat", "Tuner"), cv2.getTrackbarPos("Max Val", "Tuner")

    # Dual Mask Logic
    mask1 = cv2.inRange(hsv, np.array([lh1, ls, lv]), np.array([uh1, us, uv]))
    mask2 = cv2.inRange(hsv, np.array([lh2, ls, lv]), np.array([uh2, us, uv]))
    full_mask = cv2.bitwise_or(mask1, mask2)

    # Clean up mask
    kernel = np.ones((5,5), np.uint8)
    full_mask = cv2.morphologyEx(full_mask, cv2.MORPH_OPEN, kernel)

    # Tracking (Largest Object Only)
    contours, _ = cv2.findContours(full_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        largest = max(contours, key=cv2.contourArea)
        if cv2.contourArea(largest) > 1000:
            x, y, w, h = cv2.boundingRect(largest)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

    cv2.imshow("Detection", frame)
    cv2.imshow("Combined Mask", full_mask)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('s'): # Press 's' to save current tuning
        current_vals = {"lh1": lh1, "uh1": uh1, "lh2": lh2, "uh2": uh2, "ls": ls, "lv": lv, "us": us, "uv": uv}
        save_settings(current_vals)
        print("Settings Saved to JSON!")
    elif key == 27: 
        break

cap.release()
cv2.destroyAllWindows()

# Why this is more robust:
# Hue Wrapping: It catches "Orange-Red" (near 0) and "Purple-Red" (near 180) simultaneously, preventing the mask from flickering when the object rotates.
# Median Blur Integration: The cv2.medianBlur is placed before the mask to eliminate grainy noise.
# Persistence: Your tuning isn't lost. Using the json module, the script saves your exact lighting environment setup to a local file.
# Target Isolation: By using max(contours, key=cv2.contourArea), the code ignores background noise and locks onto the largest red object in view.
